manually install torch with cuda:
```terminal
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

In [28]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as T
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.amp import autocast, GradScaler
import torch.nn.functional as F
import copy
import numpy as np
import optuna
import time
import itertools
import pandas as pd

import optuna

### PyTorch with CUDA

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print("Using device:", device)

Using device: cuda:0


### Get CIFAR10 data

In [ ]:
# full data
# TRAINING_SIZE = 49000
# VALIDATION_SIZE = 1000

# develop data
TRAINING_SIZE = 5000
VALIDATION_SIZE = 1000
BATCH_SIZE = 512
NUM_WORKERS = 12

iters_per_epoch = np.ceil(TRAINING_SIZE / BATCH_SIZE)

RANDOM_SEED = 1
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [31]:
transform_train = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_val_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

cifar10_train = dset.CIFAR10('datasets', train=True, download=True, transform=transform_train)
# cifar10_val = dset.CIFAR10('datasets', train=True, download=True, transform=transform_val_test)
cifar10_test = dset.CIFAR10('datasets', train=False, download=True, transform=transform_val_test)

indices = np.arange(len(cifar10_train))
np.random.shuffle(indices)
train_idx = indices[:TRAINING_SIZE]
val_idx = indices[TRAINING_SIZE:TRAINING_SIZE + VALIDATION_SIZE]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

loader_train = DataLoader(
    cifar10_train,
    batch_size=BATCH_SIZE,
    sampler=train_sampler,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True
)

loader_val = DataLoader(
    cifar10_train,
    batch_size=BATCH_SIZE,
    sampler=val_sampler,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True
)

print(f"Train batches: {len(loader_train)}, Val batches: {len(loader_val)}")
print(f"Results in {iters_per_epoch} iterations per epoch.")

Train batches: 10, Val batches: 2
Results in 10.0 iterations per epoch.


### Model and Hyperparameter Training

In [32]:
def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    num_epochs=10,
    criterion=None,
    device=None,
    print_every=20,
    patience=3,
    accumulation_steps=1,
    scheduler=None
):
    """
    Optimized training function with AMP, early stopping, gradient accumulation, 
    best model saving, and optional scheduler.
    """
    device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    criterion = criterion or F.cross_entropy
    optimizer = optimizer
    scaler = GradScaler()

    best_val_acc = -float('inf')
    best_model = None
    epochs_no_improve = 0

    train_accs, val_accs, lrs, epoch_times = [], [], [], []
    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad() if accumulation_steps == 1 else None

            with autocast("cuda"):
                outputs = model(x)
                loss = criterion(outputs, y) / accumulation_steps  # scale loss for accumulation

            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            running_loss += loss.item() * accumulation_steps
            if (i + 1) % print_every == 0:
                print(f"Epoch {epoch+1}, Iter {i+1}, Avg Loss: {running_loss/print_every:.4f}")
                running_loss = 0.0

        # Step scheduler at epoch end
        if scheduler is not None:
            scheduler.step()
            lrs.append(scheduler.get_last_lr()[0])

        # Validation check
        val_acc = get_accuracy(val_loader, model, f"Validation Epoch {epoch+1}", device)
        val_accs.append(val_acc)
        
        train_acc = get_accuracy(train_loader, model, f"Training Epoch {epoch+1}", device=device)
        train_accs.append(train_acc)

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = copy.deepcopy(model)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs.")
                break

    total_time = time.time() - start_time

    return {
        "model": best_model if best_model else model,
        "train_accs": train_accs,
        "val_accs": val_accs,
        "lrs": lrs,
        "epoch_times": epoch_times,
        "total_time": total_time
    }


def get_accuracy(loader, model, name="Validation", device=device, debug=True):
    """
    Compute accuracy of the model on a given DataLoader.
    """
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            with autocast("cuda"):
                scores = model(x)
            preds = scores.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = correct / total
    if debug:
        print(f"{name} Accuracy: {100*acc:.2f}%")
    return acc

In [33]:
def hyperparam_training(learn_rate, epochs):

    ch1 = 16
    ch2 = 64
    ch3 = 32
    model = nn.Sequential(
        nn.Conv2d(3, ch1, 5, padding=2),
        nn.BatchNorm2d(ch1),
        nn.ReLU(),

        nn.Conv2d(ch1, ch2, 3, padding=1),
        nn.BatchNorm2d(ch2),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Conv2d(ch2, ch3, 3, padding=1),
        nn.BatchNorm2d(ch3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Flatten(),
        nn.Linear(ch3 * 8 * 8, 128),
        nn.ReLU(),
        nn.Dropout(0.25),
        nn.Linear(128, 10)
    ).to(device)

    weight_decay = 2.075612963823467e-06
    optimizer = torch.optim.AdamW(model.parameters(), lr=learn_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    training_result = train_model(
        model=model,
        train_loader=loader_train,
        val_loader=loader_val,
        num_epochs=epochs,
        device=device,
        print_every=len(loader_train) // 4,
        patience=2,
        optimizer=optimizer,
        scheduler=scheduler,
    )
    trained_model = training_result["model"]
    
    val_acc = get_accuracy(loader_val, trained_model, device=device, debug=False)

    return {
        "lr": learn_rate,
        "epochs": epochs,
        "val_acc": val_acc,
        "train_accs": training_result["train_accs"],
        "val_accs": training_result["val_accs"],
        "runtime": training_result["total_time"],
    }

## Search Algorithms

In [34]:
NUM_TRIALS = 25
NUM_PER_DIM = np.sqrt(NUM_TRIALS).astype(int)
LR_MIN = 1e-6
LR_MAX = 1e-2
EPOCHS_MIN = 5
EPOCHS_MAX = 25

print(f"{NUM_TRIALS} trials will be run, {NUM_PER_DIM} per dimension.")

25 trials will be run, 5 per dimension.


### Hidden Results

In [ ]:
HIDDEN_TRIALS = 200
HIDDEN_TESTS_PER_DIM = np.sqrt(HIDDEN_TRIALS).astype(int)

def run_hidden_results():
    lr_values = np.logspace(np.log10(LR_MIN), np.log10(LR_MAX), HIDDEN_TESTS_PER_DIM)
    epochs_values = np.linspace(EPOCHS_MIN, EPOCHS_MAX, HIDDEN_TESTS_PER_DIM, dtype=int)

    results = []
    trial = 1
    for lr, ep in itertools.product(lr_values, epochs_values):
        print(f"\n[Grid {trial}/{HIDDEN_TRIALS}] lr={lr:.3e}, epochs={ep}")
        res = hyperparam_training(lr, ep)
        res["trial"] = trial
        res["search"] = "grid"
        results.append(res)
        trial += 1

    return pd.DataFrame(results)

### Grid Search

In [ ]:
def run_grid_search():
    lr_values = np.logspace(np.log10(LR_MIN), np.log10(LR_MAX), NUM_PER_DIM)
    epochs_values = np.linspace(EPOCHS_MIN, EPOCHS_MAX, NUM_PER_DIM, dtype=int)

    results = []
    trial = 1
    for lr, ep in itertools.product(lr_values, epochs_values):
        print(f"\n[Grid {trial}/{NUM_PER_DIM}] lr={lr:.3e}, epochs={ep}")
        res = hyperparam_training(lr, ep)
        res["trial"] = trial
        res["search"] = "grid"
        results.append(res)
        trial += 1

    return pd.DataFrame(results)

### Random Search

In [ ]:
def run_random_search():
    lr_values = 10 ** np.random.uniform(np.log10(LR_MIN), np.log10(LR_MAX), NUM_TRIALS)
    epochs_values = np.random.randint(EPOCHS_MIN, EPOCHS_MAX+1, NUM_TRIALS)

    results = []
    trial = 1
    for lr, ep in zip(lr_values, epochs_values):
        print(f"\n[Random {trial}/{NUM_TRIALS}] lr={lr:.3e}, epochs={ep}")
        res = hyperparam_training(lr, ep)
        res["trial"] = trial
        res["search"] = "random"
        results.append(res)
        trial += 1
        
    return pd.DataFrame(results)

### Bayesian Search

In [38]:
def objective(trial):
    lr = trial.suggest_float("learn_rate", LR_MIN, LR_MAX, log=True)
    epochs = trial.suggest_int("epochs", EPOCHS_MIN, EPOCHS_MAX)

    results = hyperparam_training(lr, epochs)
    return results["val_acc"]

In [ ]:
def run_bayesian_search():
    study_name = "bayesian_optimization"
    storage_name = f"sqlite:///{study_name}.db"

    # Reset study
    summaries = optuna.study.get_all_study_summaries(storage=storage_name)
    if study_name in [s.study_name for s in summaries]:
        optuna.delete_study(study_name=study_name, storage=storage_name)
        print(f"Deleted existing study: {study_name}")

    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction="maximize",
    )

    study.optimize(objective, n_trials=NUM_TRIALS)

    rows = []
    for t in study.trials:
        row = {
            "trial": t.number + 1,
            "lr": t.params.get("learn_rate", None),
            "epochs": t.params.get("epochs", None),
            "val_acc": t.value,
            "search": "bayesian"
        }
        # todo: get time for each trial.
        rows.append(row)

    return pd.DataFrame(rows)

## Results

In [ ]:
df_hidden = run_hidden_results()
df_hidden.to_csv("results/hidden.csv")


[Grid 1/25] lr=1.000e-06, epochs=5
Epoch 1, Iter 2, Avg Loss: 2.3338
Epoch 1, Iter 4, Avg Loss: 2.3141
Epoch 1, Iter 6, Avg Loss: 2.3327
Epoch 1, Iter 8, Avg Loss: 2.3239
Epoch 1, Iter 10, Avg Loss: 2.3298
Validation Epoch 1 Accuracy: 12.50%
Training Epoch 1 Accuracy: 14.02%
Epoch 2, Iter 2, Avg Loss: 2.3235
Epoch 2, Iter 4, Avg Loss: 2.3187
Epoch 2, Iter 6, Avg Loss: 2.3429
Epoch 2, Iter 8, Avg Loss: 2.3174
Epoch 2, Iter 10, Avg Loss: 2.3338
Validation Epoch 2 Accuracy: 12.30%
Training Epoch 2 Accuracy: 13.22%
Epoch 3, Iter 2, Avg Loss: 2.3358
Epoch 3, Iter 4, Avg Loss: 2.2999
Epoch 3, Iter 6, Avg Loss: 2.3029
Epoch 3, Iter 8, Avg Loss: 2.3228
Epoch 3, Iter 10, Avg Loss: 2.3165
Validation Epoch 3 Accuracy: 14.00%
Training Epoch 3 Accuracy: 13.06%
Epoch 4, Iter 2, Avg Loss: 2.3191
Epoch 4, Iter 4, Avg Loss: 2.3099
Epoch 4, Iter 6, Avg Loss: 2.3197
Epoch 4, Iter 8, Avg Loss: 2.3102
Epoch 4, Iter 10, Avg Loss: 2.3230
Validation Epoch 4 Accuracy: 13.60%
Training Epoch 4 Accuracy: 13.04%


In [ ]:

#! combine this with cell above next time

df_grid = run_grid_search()
df_grid.to_csv("results/grid.csv")


[Grid 1/25] lr=1.000e-06, epochs=5
Epoch 1, Iter 2, Avg Loss: 2.3309
Epoch 1, Iter 4, Avg Loss: 2.3178
Epoch 1, Iter 6, Avg Loss: 2.3336
Epoch 1, Iter 8, Avg Loss: 2.3334
Epoch 1, Iter 10, Avg Loss: 2.3262
Validation Epoch 1 Accuracy: 10.80%
Training Epoch 1 Accuracy: 11.34%
Epoch 2, Iter 2, Avg Loss: 2.3205
Epoch 2, Iter 4, Avg Loss: 2.3284
Epoch 2, Iter 6, Avg Loss: 2.3329
Epoch 2, Iter 8, Avg Loss: 2.3270
Epoch 2, Iter 10, Avg Loss: 2.3292
Validation Epoch 2 Accuracy: 11.40%
Training Epoch 2 Accuracy: 11.56%
Epoch 3, Iter 2, Avg Loss: 2.3171
Epoch 3, Iter 4, Avg Loss: 2.3200
Epoch 3, Iter 6, Avg Loss: 2.3200
Epoch 3, Iter 8, Avg Loss: 2.3222
Epoch 3, Iter 10, Avg Loss: 2.3201
Validation Epoch 3 Accuracy: 12.20%
Training Epoch 3 Accuracy: 10.94%
Epoch 4, Iter 2, Avg Loss: 2.3215
Epoch 4, Iter 4, Avg Loss: 2.3203
Epoch 4, Iter 6, Avg Loss: 2.3200
Epoch 4, Iter 8, Avg Loss: 2.3194
Epoch 4, Iter 10, Avg Loss: 2.3034
Validation Epoch 4 Accuracy: 10.80%
Training Epoch 4 Accuracy: 10.54%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001D44D462520>
Traceback (most recent call last):
  File "c:\Users\Jax\Coding\Optimierungsverfahren\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "c:\Users\Jax\Coding\Optimierungsverfahren\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1577, in _shutdown_workers
    self._mark_worker_as_unavailable(worker_id, shutdown=True)
  File "c:\Users\Jax\Coding\Optimierungsverfahren\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1513, in _mark_worker_as_unavailable
    assert self._workers_status[worker_id] or (
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Training Epoch 3 Accuracy: 29.84%
Epoch 4, Iter 2, Avg Loss: 1.9467
Epoch 4, Iter 4, Avg Loss: 1.9757
Epoch 4, Iter 6, Avg Loss: 1.9576
Epoch 4, Iter 8, Avg Loss: 1.9507
Epoch 4, Iter 10, Avg Loss: 1.9600
Validation Epoch 4 Accuracy: 30.80%
Training Epoch 4 Accuracy: 33.68%
Epoch 5, Iter 2, Avg Loss: 1.8995
Epoch 5, Iter 4, Avg Loss: 1.9165
Epoch 5, Iter 6, Avg Loss: 1.8843
Epoch 5, Iter 8, Avg Loss: 1.8851
Epoch 5, Iter 10, Avg Loss: 1.8894
Validation Epoch 5 Accuracy: 35.20%
Training Epoch 5 Accuracy: 36.26%
Epoch 6, Iter 2, Avg Loss: 1.8713
Epoch 6, Iter 4, Avg Loss: 1.8996
Epoch 6, Iter 6, Avg Loss: 1.8503
Epoch 6, Iter 8, Avg Loss: 1.8290
Epoch 6, Iter 10, Avg Loss: 1.8027
Validation Epoch 6 Accuracy: 35.80%
Training Epoch 6 Accuracy: 36.44%
Epoch 7, Iter 2, Avg Loss: 1.8254
Epoch 7, Iter 4, Avg Loss: 1.8632
Epoch 7, Iter 6, Avg Loss: 1.8646
Epoch 7, Iter 8, Avg Loss: 1.7775
Epoch 7, Iter 10, Avg Loss: 1.7826
Validation Epoch 7 Accuracy: 36.10%
Training Epoch 7 Accuracy: 38.66%
Ep

In [74]:
df_random = run_random_search()
df_random.to_csv("results/random.csv")

#! combine this with cell above next time

# df_bayesian = run_bayesian_search()
# df_bayesian.to_csv("results/bayesian.csv")


[Random 1/25] lr=4.824e-06, epochs=6
Epoch 1, Iter 2, Avg Loss: 2.3350
Epoch 1, Iter 4, Avg Loss: 2.3512
Epoch 1, Iter 6, Avg Loss: 2.3294
Epoch 1, Iter 8, Avg Loss: 2.3252
Epoch 1, Iter 10, Avg Loss: 2.3337
Validation Epoch 1 Accuracy: 11.40%
Training Epoch 1 Accuracy: 10.36%
Epoch 2, Iter 2, Avg Loss: 2.3301
Epoch 2, Iter 4, Avg Loss: 2.3255
Epoch 2, Iter 6, Avg Loss: 2.3228
Epoch 2, Iter 8, Avg Loss: 2.3213
Epoch 2, Iter 10, Avg Loss: 2.3136
Validation Epoch 2 Accuracy: 10.60%
Training Epoch 2 Accuracy: 10.88%
Epoch 3, Iter 2, Avg Loss: 2.3156
Epoch 3, Iter 4, Avg Loss: 2.3165
Epoch 3, Iter 6, Avg Loss: 2.3192
Epoch 3, Iter 8, Avg Loss: 2.3216
Epoch 3, Iter 10, Avg Loss: 2.2986
Validation Epoch 3 Accuracy: 12.00%
Training Epoch 3 Accuracy: 11.40%
Epoch 4, Iter 2, Avg Loss: 2.3169
Epoch 4, Iter 4, Avg Loss: 2.3152
Epoch 4, Iter 6, Avg Loss: 2.3141
Epoch 4, Iter 8, Avg Loss: 2.3023
Epoch 4, Iter 10, Avg Loss: 2.3085
Validation Epoch 4 Accuracy: 11.60%
Training Epoch 4 Accuracy: 11.00

In [84]:

#! combine this with cell above next time

df_bayesian = run_bayesian_search()
df_bayesian.to_csv("results/bayesian.csv")

[I 2025-12-01 00:38:42,442] A new study created in RDB with name: bayesian_optimization


Deleted existing study: bayesian_optimization
Epoch 1, Iter 2, Avg Loss: 2.3374
Epoch 1, Iter 4, Avg Loss: 2.3367
Epoch 1, Iter 6, Avg Loss: 2.3370
Epoch 1, Iter 8, Avg Loss: 2.3245
Epoch 1, Iter 10, Avg Loss: 2.3289
Validation Epoch 1 Accuracy: 8.40%
Training Epoch 1 Accuracy: 9.60%
Epoch 2, Iter 2, Avg Loss: 2.3162
Epoch 2, Iter 4, Avg Loss: 2.3225
Epoch 2, Iter 6, Avg Loss: 2.3264
Epoch 2, Iter 8, Avg Loss: 2.3117
Epoch 2, Iter 10, Avg Loss: 2.3205
Validation Epoch 2 Accuracy: 10.20%
Training Epoch 2 Accuracy: 10.02%
Epoch 3, Iter 2, Avg Loss: 2.3091
Epoch 3, Iter 4, Avg Loss: 2.3125
Epoch 3, Iter 6, Avg Loss: 2.3151
Epoch 3, Iter 8, Avg Loss: 2.3093
Epoch 3, Iter 10, Avg Loss: 2.2999
Validation Epoch 3 Accuracy: 11.10%
Training Epoch 3 Accuracy: 11.64%
Epoch 4, Iter 2, Avg Loss: 2.2899
Epoch 4, Iter 4, Avg Loss: 2.2973
Epoch 4, Iter 6, Avg Loss: 2.3000
Epoch 4, Iter 8, Avg Loss: 2.2791
Epoch 4, Iter 10, Avg Loss: 2.2844
Validation Epoch 4 Accuracy: 13.30%
Training Epoch 4 Accuracy:

[I 2025-12-01 00:38:47,883] Trial 0 finished with value: 0.141 and parameters: {'learn_rate': 5.1644759858707125e-06, 'epochs': 11}. Best is trial 0 with value: 0.141.


Epoch 1, Iter 2, Avg Loss: 3.0359
Epoch 1, Iter 4, Avg Loss: 3.1757
Epoch 1, Iter 6, Avg Loss: 2.4305
Epoch 1, Iter 8, Avg Loss: 2.1955
Epoch 1, Iter 10, Avg Loss: 2.0724
Validation Epoch 1 Accuracy: 23.50%
Training Epoch 1 Accuracy: 24.02%
Epoch 2, Iter 2, Avg Loss: 2.0245
Epoch 2, Iter 4, Avg Loss: 2.0446
Epoch 2, Iter 6, Avg Loss: 1.9827
Epoch 2, Iter 8, Avg Loss: 1.9669
Epoch 2, Iter 10, Avg Loss: 1.9725
Validation Epoch 2 Accuracy: 26.70%
Training Epoch 2 Accuracy: 28.50%
Epoch 3, Iter 2, Avg Loss: 1.9518
Epoch 3, Iter 4, Avg Loss: 1.9077
Epoch 3, Iter 6, Avg Loss: 1.9037
Epoch 3, Iter 8, Avg Loss: 1.8962
Epoch 3, Iter 10, Avg Loss: 1.8614
Validation Epoch 3 Accuracy: 29.80%
Training Epoch 3 Accuracy: 32.50%
Epoch 4, Iter 2, Avg Loss: 1.8435
Epoch 4, Iter 4, Avg Loss: 1.8554
Epoch 4, Iter 6, Avg Loss: 1.8045
Epoch 4, Iter 8, Avg Loss: 1.7491
Epoch 4, Iter 10, Avg Loss: 1.7448
Validation Epoch 4 Accuracy: 33.10%
Training Epoch 4 Accuracy: 35.90%
Epoch 5, Iter 2, Avg Loss: 1.7515
Ep

[I 2025-12-01 00:38:53,357] Trial 1 finished with value: 0.409 and parameters: {'learn_rate': 0.0026297435968120537, 'epochs': 9}. Best is trial 1 with value: 0.409.


...

[I 2025-12-01 00:41:28,965] Trial 22 finished with value: 0.503 and parameters: {'learn_rate': 0.0009134709936713705, 'epochs': 20}. Best is trial 11 with value: 0.511.


Training Epoch 15 Accuracy: 54.86%
Early stopping triggered after 15 epochs.
Epoch 1, Iter 2, Avg Loss: 2.3712
Epoch 1, Iter 4, Avg Loss: 2.2373
Epoch 1, Iter 6, Avg Loss: 2.1833
Epoch 1, Iter 8, Avg Loss: 2.1200
Epoch 1, Iter 10, Avg Loss: 2.0402
Validation Epoch 1 Accuracy: 23.10%
Training Epoch 1 Accuracy: 25.28%
Epoch 2, Iter 2, Avg Loss: 1.9862
Epoch 2, Iter 4, Avg Loss: 1.9405
Epoch 2, Iter 6, Avg Loss: 1.9107
Epoch 2, Iter 8, Avg Loss: 1.9025
Epoch 2, Iter 10, Avg Loss: 1.8770
Validation Epoch 2 Accuracy: 32.80%
Training Epoch 2 Accuracy: 34.32%
Epoch 3, Iter 2, Avg Loss: 1.8124
Epoch 3, Iter 4, Avg Loss: 1.8030
Epoch 3, Iter 6, Avg Loss: 1.7563
Epoch 3, Iter 8, Avg Loss: 1.7764
Epoch 3, Iter 10, Avg Loss: 1.7289
Validation Epoch 3 Accuracy: 36.90%
Training Epoch 3 Accuracy: 38.64%
Epoch 4, Iter 2, Avg Loss: 1.6976
Epoch 4, Iter 4, Avg Loss: 1.7198
Epoch 4, Iter 6, Avg Loss: 1.6709
Epoch 4, Iter 8, Avg Loss: 1.6475
Epoch 4, Iter 10, Avg Loss: 1.5846
Validation Epoch 4 Accuracy: 

[I 2025-12-01 00:41:35,562] Trial 23 finished with value: 0.477 and parameters: {'learn_rate': 0.0010820097183441267, 'epochs': 22}. Best is trial 11 with value: 0.511.


Epoch 1, Iter 2, Avg Loss: 3.2211
Epoch 1, Iter 4, Avg Loss: 4.2995
Epoch 1, Iter 6, Avg Loss: 2.9509
Epoch 1, Iter 8, Avg Loss: 2.3564
Epoch 1, Iter 10, Avg Loss: 2.2785
Validation Epoch 1 Accuracy: 18.40%
Training Epoch 1 Accuracy: 18.20%
Epoch 2, Iter 2, Avg Loss: 2.1974
Epoch 2, Iter 4, Avg Loss: 2.1201
Epoch 2, Iter 6, Avg Loss: 2.1042
Epoch 2, Iter 8, Avg Loss: 2.0853
Epoch 2, Iter 10, Avg Loss: 2.0942
Validation Epoch 2 Accuracy: 18.70%
Training Epoch 2 Accuracy: 20.66%
Epoch 3, Iter 2, Avg Loss: 2.0570
Epoch 3, Iter 4, Avg Loss: 2.0155
Epoch 3, Iter 6, Avg Loss: 2.0168
Epoch 3, Iter 8, Avg Loss: 2.0114
Epoch 3, Iter 10, Avg Loss: 2.0398
Validation Epoch 3 Accuracy: 24.30%
Training Epoch 3 Accuracy: 24.92%
Epoch 4, Iter 2, Avg Loss: 1.9840
Epoch 4, Iter 4, Avg Loss: 1.9691
Epoch 4, Iter 6, Avg Loss: 1.9446
Epoch 4, Iter 8, Avg Loss: 1.9181
Epoch 4, Iter 10, Avg Loss: 1.9290
Validation Epoch 4 Accuracy: 27.70%
Training Epoch 4 Accuracy: 28.84%
Epoch 5, Iter 2, Avg Loss: 1.8971
Ep

[I 2025-12-01 00:41:46,512] Trial 24 finished with value: 0.409 and parameters: {'learn_rate': 0.0035921067254284316, 'epochs': 18}. Best is trial 11 with value: 0.511.


## Visualizations

In [59]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import os

sns.set_theme(context="talk", style="whitegrid")
# sns.set_theme()

FIGSIZE = {
    "full": (6.4, 3.6),
    "half": (3.1, 1.8),
    "tall": (5.0, 4.0),
    "wide": (7.0, 3.5),
}

plt.rcParams.update({
    # --- Figure layout ---
    "figure.figsize": FIGSIZE["full"],      # 16:9 aspect ratio (full width)
    "figure.dpi": 200,                 
    "savefig.dpi": 300,                

    # --- Font sizes ---
    "font.size": 10,                   # match Beamer base font
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,

    # --- Fonts ---
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Fira Sans"],
    "text.latex.preamble": r"""
        \usepackage{FiraSans}
        \usepackage{sfmath}
        \renewcommand*\familydefault{\sfdefault}
    """,

    # --- Axes and grid ---
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.grid": True,
    "grid.linestyle": ":",
    "grid.alpha": 0.3,

    # --- Save options ---
    # "savefig.bbox": "tight",
    "savefig.pad_inches": 0.02,
})

In [52]:
def savefig(name, ext="pdf", imag_dir="images", tight=True, transparent=True, dpi=300):
    """
    Save the current matplotlib figure in a clean, consistent format.

    Parameters
    ----------
    name : str
        Base filename (no extension).
    ext : str, optional
        File extension, e.g. 'pdf', 'png', or 'svg'. Default is 'pdf'.
    tight : bool, optional
        Apply tight_layout() before saving. Default is True.
    transparent : bool, optional
        Use transparent background (useful for LaTeX Beamer overlays). Default is False.
    dpi : int, optional
        Dots per inch for raster formats (e.g., PNG). Default is 300.
    """

    # Ensure output directory exists
    os.makedirs(imag_dir, exist_ok=True)
    
    path = os.path.join(imag_dir, f"{name}.{ext}")
    plt.savefig(path, dpi=dpi, bbox_inches="tight", transparent=transparent)

    plt.close()
    print(f"💾 Saved: {path}")

def plot_contour(df, filename, df_hidden):
    """
    df         → DataFrame der Suchmethode (z.B. df_grid / df_random / df_bayesian)
    filename   → Name für savefig()
    df_hidden  → vollständiger Hyperparameter-Grid (für Contour)
    """

    # build lr/epoch grid
    lr_vals = np.sort(df_hidden["lr"].unique())
    epoch_vals = np.sort(df_hidden["epochs"].unique())

    Z = (
        df_hidden
        .pivot(index="epochs", columns="lr", values="val_acc")
        .reindex(index=epoch_vals, columns=lr_vals)
    )

    X, Y = np.meshgrid(lr_vals, epoch_vals)

    # Number of contour levels = reasonable (not N points!)
    n_levels = df_hidden.shape[0] // 5

    plt.figure(constrained_layout=True)

    # no white contour edges → linewidths=0
    cs = plt.contourf(
        X, Y, Z.values,
        levels=n_levels,
        cmap="viridis",
    )
    plt.xscale("log")
    plt.xlabel("Lernrate")
    plt.ylabel("Epochen")

    # Colorbar formatting
    cbar = plt.colorbar(cs, label="Accuracy")
    cbar.ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))

    # Epoch axis integer format
    plt.gca().yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

    # Search points
    plt.scatter(df["lr"], df["epochs"], color="black", label="Suchpunkte")

    # Best point
    best = df.loc[df["val_acc"].idxmax()]
    plt.scatter(
        best["lr"], best["epochs"],
        color="red",
        s=60,
        label=f"Bestes Ergebnis = {best['val_acc']:.2f}"
    )

    plt.legend()
    savefig(filename)
    plt.show()

In [53]:
df_hidden = pd.read_csv("results/hidden.csv", index_col=0)
df_grid = pd.read_csv("results/grid.csv", index_col=0)
df_random = pd.read_csv("results/random.csv", index_col=0)
df_bayesian = pd.read_csv("results/bayesian.csv", index_col=0)

In [54]:
plot_contour(df_grid, "contour_grid", df_hidden)
plot_contour(df_random, "contour_random", df_hidden)
plot_contour(df_bayesian, "contour_bayesian", df_hidden)

💾 Saved: images\contour_grid.pdf
💾 Saved: images\contour_random.pdf
💾 Saved: images\contour_bayesian.pdf


In [67]:
# Prepare combined dataframe
df_all = pd.concat([df_grid, df_random, df_bayesian], ignore_index=True)

# Compute global max and band
global_max = df_all["val_acc"].max()
LOWER_BOUND = 0.9
lower_band = global_max * LOWER_BOUND

# Create FacetGrid with columns per method
g = sns.FacetGrid(
    df_all,
    col="search",
    col_wrap=3,
    height=4,
    aspect=1.2,
    sharex=True,
    sharey=True
)

# TODO: change counter to trial next time
# Scatter plot
g.map_dataframe(
    sns.scatterplot,
    x="counter",
    y="val_acc",
    # s=25,
    # alpha=0.7
)

# Add rolling max line + hspan
for ax, method in zip(g.axes.flat, df_all["search"].unique()):
    df_m = df_all[df_all["search"] == method].sort_values("counter")
    
    # Rolling max
    ax.plot(
        df_m["counter"], 
        df_m["val_acc"].cummax(), 
        color="orange", 
        linestyle="--",
        label="Aktueller Bestwert")

    # top band
    ax.axhspan(lower_band, global_max, 
            color="grey", 
            alpha=0.15, 
            label=f"Top {int(np.round((1-LOWER_BOUND)*100))}% Band"
    )
    
    # Labels
    ax.set_title(method)
    ax.set_xlabel("Trial")
    ax.set_ylabel("Accuracy")

# handles, labels = ax.get_legend_handles_labels()
# g.fig.legend(
#     handles,
#     labels,
#     loc="lower center",
#     ncol=3,
#     frameon=False
# )

plt.tight_layout()
savefig("comparison")
plt.show()

💾 Saved: images\comparison.pdf
